In [1]:
import pandas as pd
from dataloader import loadEntityData

df = loadEntityData('./data/HackerForumEntities.csv')

Index(['ID', 'Username', 'PostDate', 'ThreadTitle', 'PostContent',
       'PostNumber', '__index_level_0__', 'Entities'],
      dtype='object')

In [15]:
# TODO Do data analysis stuff

# Unpack and count

pandas.core.series.Series